In [5]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import shutil
import kagglehub
import torch
from pyspark.sql import SparkSession

dataset_path = os.path.join(ROOT_DIR, 'neural', 'datasets', 'spark', 'test_1')
os.makedirs(dataset_path, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import socket

driver_host = socket.gethostbyname(socket.gethostname())
print(driver_host)
driver_host = socket.gethostbyname("spark-master")
print(driver_host)
driver_host = "producer"
os.environ["PYSPARK_SUBMIT_ARGS"] = f"""
--master spark://spark-master:7077
--conf spark.driver.host={driver_host}
--conf spark.driver.port=45555
--conf spark.executor.memory=1G
--conf spark.executor.cores=1
pyspark-shell
"""
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'  # путь к Python в контейнере
# os.environ['SPARK_HOME'] = '/opt/spark'            # путь к Spark, укажи реальный
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
import socket
socket.gethostbyname("spark-master")

172.20.0.2
192.168.32.2


'192.168.32.2'

In [6]:
def spark_app_generator(name):
    spark = SparkSession.builder \
        .master("spark://spark-master:7077") \
        .appName(name) \
         .config("spark.driver.bindAddress", "0.0.0.0") \
        .config("spark.executor.memory", "1g") \
        .config("spark.driver.memory", "1g") \
        .getOrCreate()
    return spark
spark = spark_app_generator("test_counting")

In [8]:
df = spark.range(100).toDF("number")
df.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [5]:
spark = spark_app_generator("test_square")

# Запускаем простую задачу
rdd = spark.sparkContext.parallelize([1, 2, 3, 4])

print(rdd.map(lambda x: x * x).collect())

[1, 4, 9, 16]


In [6]:
from pyspark.sql import SparkSession

spark = spark_app_generator("test_WordCountTest")
rdd = spark.sparkContext.textFile("file:////" + os.path.abspath(os.path.join(dataset_path, 'test.txt')))

counts = rdd.flatMap(lambda line: line.split(" ")) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)
result = counts.collect()
result


25/04/15 16:49:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


[('hello', 6),
 ('world', 2),
 ('spark3', 1),
 ('docker5', 1),
 ('spark', 1),
 ('docker', 1)]

## MONGO

In [1]:
!pip install pymongo pymongo-schema

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.4 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13781 sha256=229c80c0e74252cd4e75a56ca36f4c000b25f8560f8a83adfa2e6b7c63ab71e1
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273900 sha256=72f6188a9f1f3c197defeda80795d1484d5e74db7d015084df3c82c7aa9b630d
  Stored in directory: /root/.cache/pip/wheels/4f/18/8d/3800b8b1dc7a8c1954eaa48424f639b2cfc760922cc3cee479
Successfully built docopt ete3


In [7]:
import pymongo

MONGO_USER = os.environ["MONGO_USER"]
MONGO_PASS = os.environ["MONGO_PASSWORD"]
MONGO_ADDR = f"{MONGO_USER}:{MONGO_PASS}@mongodb:27017"  # :27017

maxSevSelDelay = 1 # Assume 1ms maximum server selection
client = pymongo.MongoClient(f"mongodb://{MONGO_ADDR}",
                                 serverSelectionTimeoutMS=maxSevSelDelay)

print(client.server_info())
print(client.list_database_names())
print(client.off.list_collection_names())

ServerSelectionTimeoutError: No servers found yet, Timeout: 0.001s, Topology Description: <TopologyDescription id: 67fec38c9e55c6e4c2615c7f, topology_type: Unknown, servers: [<ServerDescription ('xorze.keenetic.pro', 36026) server_type: Unknown, rtt: None>]>